## Introduction

The goal of this notebook is to show how you can read a very big file chunk by chunk.

In Talking Data competition, if you load train.csv file with a standard pd.read_csv command, the resulting DataFrame would use 11GB of memory. This is too large for most of personal computers setup but you can still read and work with whole data using pandas. 

By the end of this session you will know how to :
 
 - read a csv file in chunks using the **chunksize** argument, i.e. in slices of N rows
 - iterate though the chunks
 - use DataFrame **groupby** method to create simple average statistics
 - use **pd.DataFrame.add** to sum dataframes with different index's entries
 - use average statistics to make simple predictions
 
So let's start

In [1]:
import pandas as pd
import numpy as np

Please change the file_path so that it points to where the train file is on your system  

In [2]:
file_path = "../input/train.csv.zip"

From the hard work you have done in the first notebook you can define the best data types for each columns.

Again in this notebook we will exclude all time related columns. 

Here are the data types definition: 

In [3]:
dtypes = {
        'ip': 'uint32',
        'app': 'uint16',
        'device': 'uint16',
        'os': 'uint16',
        'channel': 'uint16',
        'is_attributed': 'uint8'
    }
cols = [f_ for f_ in dtypes.keys()]

To read data by chunks will use the chunksize argument of pd.read_csv method 

chunksize is the maximum number of rows each chunk should be made of.

pandas will read the file and give the 1st N rows then the following N rows and so on until the end of the file.


In [4]:
# chunksize is the maximum number of rows each chunk should be made of
import time
import gc
# Enable garbage collection
gc.enable()
chunksize = 10000000
start_time = time.time()
for i_chunk, df in enumerate(pd.read_csv(file_path, chunksize=chunksize, dtype=dtypes, usecols=cols)):
    print("%3d Chunks have been read in %5.1f minute" 
          % (i_chunk + 1, (time.time() - start_time) / 60))
    # Free memory by deleting the current DataFrame
    del df
    gc.collect()
    
    # Make sure we stop after 3 chunks
    if i_chunk >= 2 :
        break

  1 Chunks have been read in   0.1 minute
  2 Chunks have been read in   0.2 minute
  3 Chunks have been read in   0.3 minute


Even if you read by chunks you can still limit the number of total rows you want to access using nrows like in the following example

In [5]:
chunksize = 10000000
start_time = time.time()
# Read by chunks and limit the total number of rows accessed to the first 30 000 000 rows
for i_chunk, df in enumerate(pd.read_csv(file_path, chunksize=chunksize, dtype=dtypes, usecols=cols, nrows=30000000)):
    print("%3d Chunks have been read in %5.1f minute" 
          % (i_chunk + 1, (time.time() - start_time) / 60))
    # Free memory by deleting the current DataFrame
    del df
    gc.collect()

  1 Chunks have been read in   0.1 minute
  2 Chunks have been read in   0.2 minute
  3 Chunks have been read in   0.3 minute


You could also limit the access to the last N rows using the **skiprows** argument.

Have a look at the first notebook to find out more on this.

## Creating simple statistics

The goal of this section is to show how you can calculate the probability a given IP address will be attributed.
In other words we will try to compute :
$$P\left ( is\_attributed= 1\mid ip\_address \right )$$

To do this we have to go though all ip addresses and sum up the number of times is_attributed=1 for each ip address and count the number of occurences of each ip address.

The simplest way to do this is :

1 Read the data (here we limit to 10 million rows)

In [6]:
df = pd.read_csv(file_path, dtype=dtypes, nrows=10000000, usecols=['ip', 'is_attributed'])

In [11]:
print("Memory used for 10 million rows with only ip and is_attributed %5.2f MB"
      % (df.memory_usage().sum() / 1024 ** 2))

Memory used for 10 million rows with only ip and is_attributed 47.68 MB


2 Aggregate data by ip address using the groupby statement and using mean to calculate the average

In [14]:
ip_average = df.groupby("ip").mean()
ip_average.head(10)

,is_attributed
ip,
9,0.000000
10,0.000000
19,0.000000
20,0.000000
25,0.015625
27,0.000000
31,0.000000
32,0.000000
36,0.000000


3 To create predictions for each ip we use the map statement

In [15]:
df["predictions"] = df["ip"].map(ip_average["is_attributed"])
df["predictions"].head(10)

0    0.000754
1    0.000000
2    0.000000
3    0.001064
4    0.000000
5    0.000000
6    0.000000
7    0.000000
8    0.000000
9    0.007576
Name: predictions, dtype: float64

4 We can now check the AUC score for this prediction using sklearn metric roc_auc_score

In [16]:
from sklearn.metrics import roc_auc_score
print("AUC score for simple IP address predictions = %.6f"
      % (roc_auc_score(df["is_attributed"], df["predictions"])))

AUC score for simple IP address predictions = 0.949700


With this section you now know how to create predictions using the ip address only.

However this is on the first 10 million rows only

## Exercise

Make predictions and calculate scores using the method above for the following features : 
 - app
 - device
 - os
 - channel
 
Please don't forget to free up memory after each prediction as shown below

In [17]:
del df, ip_average
gc.collect()

55

## Creating statistics using the entire file

In the previous method, reading the full file for ip and is_attributed would not be a problem and would take up to 890MB.

However the groupby statement is very memory consuming and would certainly end up in a memory error situation.

This is where using chunks is useful

Please adapt chunksize to your own memory setup

In [74]:
# Create ip_average as an empty DataFrame
ip_average = None
start_time=time.time()
chunksize=20000000
for i_chunk, df in enumerate(pd.read_csv(file_path, chunksize=chunksize, dtype=dtypes, usecols=['ip', 'is_attributed'])):
    print("%3d Chunks have been read in %5.1f minute" 
          % (i_chunk + 1, (time.time() - start_time) / 60))
    # Make the groupby statement
    # The groupby statement uses sum and count to be able to compute averages over all samples
    the_group = df.groupby("ip").agg(['sum', 'count'])
    the_group.columns = the_group.columns.droplevel(0)
    if ip_average is None:
        ip_average = the_group
    else:
        # pandas .add method makes sure ip addresses that are not in both the_group and ip_average
        # take value of 0 before the addition takes place
        ip_average = the_group.add(ip_average, fill_value=0.0)

    # Free memory by deleting the current DataFrame
    del df, the_group
    gc.collect()
    
ip_average.head(10)

  1 Chunks have been read in   0.2 minute
  2 Chunks have been read in   0.4 minute
  3 Chunks have been read in   0.6 minute
  4 Chunks have been read in   0.8 minute
  5 Chunks have been read in   1.1 minute
  6 Chunks have been read in   1.3 minute
  7 Chunks have been read in   1.5 minute
  8 Chunks have been read in   1.7 minute
  9 Chunks have been read in   2.0 minute
 10 Chunks have been read in   2.1 minute


,sum,count
ip,,
1,9.0,47.0
5,0.0,24.0
6,2.0,1454.0
9,6.0,4029.0
10,3.0,1180.0
19,4.0,846.0
20,4.0,5971.0
25,2.0,446.0
27,8.0,5007.0


Now create the average per ip address

In [75]:
ip_average['average'] = ip_average['sum'] / ip_average['count'] 

Now we need to go through the file again to create the predictions

In [76]:
# Create ip_average as an empty DataFrame
start_time=time.time()
# Create place holders for target and predictions to be able to compute the AUC score once the process has completed
target = None
predictions = None 
for i_chunk, df in enumerate(pd.read_csv(file_path, chunksize=chunksize, dtype=dtypes, usecols=['ip', 'is_attributed'])):
    print("%3d Chunks have been read in %5.1f minute" 
          % (i_chunk + 1, (time.time() - start_time) / 60))
    if target is None:
        target = df['is_attributed'].values
        predictions = df['ip'].map(ip_average['average']).values
    else:
        target = np.hstack((target, df['is_attributed'].values))
        predictions = np.hstack((predictions, df['ip'].map(ip_average['average']).values))
        
    # Free memory by deleting the current DataFrame
    del df
    gc.collect()

  1 Chunks have been read in   0.2 minute
  2 Chunks have been read in   0.4 minute
  3 Chunks have been read in   0.7 minute
  4 Chunks have been read in   0.9 minute
  5 Chunks have been read in   1.2 minute
  6 Chunks have been read in   1.4 minute
  7 Chunks have been read in   1.7 minute
  8 Chunks have been read in   1.9 minute
  9 Chunks have been read in   2.2 minute
 10 Chunks have been read in   2.3 minute


Display AUC score for this simple prediction on training dataset

Please note this may take some time

In [79]:
print("AUC score of predictions using ip on the whole dataset = %.6f"
      % (roc_auc_score(target, predictions)))

AUC score of predictions using ip on the whole dataset = 0.825532


## Exercise

Please use previous code to create predictions for :
 - app
 - device
 - channel
 - os
 
And give us your results.